In [3]:
"""
R001_Fall_2023_AI_Reproducibility__
|
R001_Extract_Citing_Paper_Details_from_S2GA.ipynb
Created on Thu Sep 21 14:36:12 2023
@author: Rochana Obadage
"""

'\nR001_Fall_2023_AI_Reproducibility__\n|\nR001_Extract_Citing_Paper_Details_from_S2GA.ipynb\nCreated on Thu Sep 21 14:36:12 2023\n@author: Rochana Obadage\n'

# ``Extract citing paper details from Semantic Scholar Graph API (S2GA)`` 

### `dependencies`
> <font face="consolas" color='#000080'><b>1. python:</b><br> 
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;3.8.17&emsp;&emsp;|&emsp;&emsp;3.9.13&emsp;&emsp;|&emsp;&emsp;3.10.8<br><br>
<b>2. Jupyter Notebook:</b><br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;notebook server: 6.5.4<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Kernel Information: Python 3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]<br><br>
<b>3. conda installations:</b><br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;pandas==2.0.3<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Beatifulsoup4==4.11.1<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Selenium==4.11.2<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge selenium<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;webdriver-manager=4.0.0<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge webdriver-manager<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;bibtexparser-1.4.0<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge bibtexparser!<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;pdfminer.six-20221105<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge pdfminer.six<br>

</font>


In [6]:
# !python --version

In [7]:
# !where python

In [8]:
# !where pip

In [11]:
# !python -m ipykernel install --user --name=<"kernel name"> --display-name=<"My Official Kernel Name">
# !jupyter kernelspec list

In [18]:
from bs4 import BeautifulSoup, NavigableString, Tag

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode
from pdfminer.high_level import extract_pages

import pandas as pd
import requests
import json
import urllib
import time
import string
import os
import shutil
import math
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import bibtexparser

## ``DOI pre processing``

In [19]:
df_doi = pd.read_excel(r'../data/RS_ALL_IN_ONE_metadata.xlsx',sheet_name="Sheet1")

pd.set_option('display.max_columns', None)
df_doi.head(10)

,key_for_all_RS,title_rs,authors_rs,article_type,keywords,abstract,pdf_url_rs,code_url_rs,org_paper_url,doi_rs,code_doi_rs,doi_org,s2ga_key,citation_count_org,citing_paper_count,total_context_count_for_labelling,venue_name_rs,challenge_year_rs,review_url_rs,journal_rs,volume_journal_rs,issue_journal_rs,published_year_journal_rs,domain,entry_type,rs_score,total_claims_or_attempts,results_threshold,rs_score_comment,rs_comment
0,RS_001_MLRC_2022_01,[Re] $\mathcal{G}$-Mixup: Graph Data Augmentat...,"['Ermin Omeragic', 'Vuk ÄuranoviÄ‡']",Replication,"['rescience c', 'machine learning', 'data augm...",Scope of Reproducibility\nThis paper presents ...,/pdf/8c87cb1e84e1482826c40a3b0c43928eaef747f3.pdf,NaN,https://proceedings.mlr.press/v162/han22c.html,https://www.doi.org/10.5281/zenodo.8173650,NaN,https://doi.org/10.48550/arXiv.2202.07179,arXiv:2202.07179,76,57,141,MLRC,2022,https://openreview.net/forum?id=XxUIomN-ndH,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,0.43,8,Not given,3.5/8,Results — Due to many missing implementation d...
1,RS_002_MLRC_2022_02,[Re] Exact Feature Distribution Matching for A...,"['Mert Erkol', 'Furkan KÄ±nlÄ±', 'BarÄ±ÅŸ Ã–zc...",Replication,"['feature distribution matching', 'style trans...",Reproducibility Summary:\n\nIn this reproducib...,/pdf/7bcb577c2a46db29c48234a5b72368053c7ebed3.pdf,NaN,https://openaccess.thecvf.com/content/CVPR2022...,https://www.doi.org/10.5281/zenodo.8173652,NaN,10.1109/CVPR52688.2022.00787,10.1109/CVPR52688.2022.00787,65,38,160,MLRC,2022,https://openreview.net/forum?id=a5_hbZf0NB&not...,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,5,Not given,Although our experimental results are not\nide...,Results — We have reproduced the experiments d...
2,RS_003_MLRC_2022_03,[Re] End-to-end Algorithm Synthesis with Recur...,"['Sean Michael McLeish', 'Long Tran-Thanh']",Replication,"['rescience c', 'Algorithmic Reasoning', 'Deep...","Scope of Reproducibility:\nIn this report, we ...",/pdf/07d5d68b5873d779bd1fd8c95b9767cb57fe0bc4.pdf,NaN,https://arxiv.org/pdf/2202.05826.pdf,https://www.doi.org/10.5281/zenodo.8173654,NaN,https://doi.org/10.48550/arXiv.2202.05826,arXiv:2202.05826,15,15,34,MLRC,2022,https://openreview.net/forum?id=WaZB4pUVTi,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,3,Not given,apart from perturbation testing for which we p...,Results — We verify the authors’ claims by rep...
3,RS_004_MLRC_2022_04,[Re] Label-Free Explainability for Unsupervise...,"['Eric Langezaal', 'Jesse Belleman', 'Joeri No...",Replication,"['Reproducibility', 'label-free', 'unsupervise...",Scope of Reproducibility â€” This study is an ...,/pdf/7fde4f12c675150699fc376cd097d5b9dad0b4d4.pdf,NaN,https://proceedings.mlr.press/v162/crabbe22a.html,https://www.doi.org/10.5281/zenodo.8173656,NaN,https://doi.org/10.48550/arXiv.2203.01928,arXiv:2203.01928,12,6,16,MLRC,2022,https://openreview.net/forum?id=bBVZ3pY4z8p,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,3,Not given,"Our results differ in some minor aspects, but ...",Results — We find that the three main claims o...
4,RS_005_MLRC_2022_05,[Re] Exploring the Representation of Word Mean...,"['Matteo Brivio', 'Cagri Coltekin']",Replication,"['BERT', 'fastText', 'embeddings', 'language m...",This report summarizes our efforts to reproduc...,/pdf/78430c5af33bc892d852b49a5a6e93abeb314c6f.pdf,NaN,https://aclanthology.org/2021.acl-long.281/,https://www.doi.org/10.5281/zenodo.8173658,NaN,10.18653/v1/2021.acl-long.281,10.18653/v1/2021.acl-long.281,7,6,10,MLRC,2022,https://openreview.net/forum?id=Od5dD58libt,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,0.75,4,Not given,We only manage to partially reproduce the orig...,Results — We only manage to partially reproduc...
5,RS_006_MLRC_2022_06,[Re] Intriguing Properties of Contrastive Losses,"['Luca Marini', 'Mohamad Nabeel', 'Alexandre L...",Replication,"['self-supervised learning', 'contrastive lear...",Reproducibility Summary\n\nScope of Reproducib...,/pdf

In [20]:
df_doi.head(10)

,key_for_all_RS,title_rs,authors_rs,article_type,keywords,abstract,pdf_url_rs,code_url_rs,org_paper_url,doi_rs,code_doi_rs,doi_org,s2ga_key,citation_count_org,citing_paper_count,total_context_count_for_labelling,venue_name_rs,challenge_year_rs,review_url_rs,journal_rs,volume_journal_rs,issue_journal_rs,published_year_journal_rs,domain,entry_type,rs_score,total_claims_or_attempts,results_threshold,rs_score_comment,rs_comment
0,RS_001_MLRC_2022_01,[Re] $\mathcal{G}$-Mixup: Graph Data Augmentat...,"['Ermin Omeragic', 'Vuk ÄuranoviÄ‡']",Replication,"['rescience c', 'machine learning', 'data augm...",Scope of Reproducibility\nThis paper presents ...,/pdf/8c87cb1e84e1482826c40a3b0c43928eaef747f3.pdf,NaN,https://proceedings.mlr.press/v162/han22c.html,https://www.doi.org/10.5281/zenodo.8173650,NaN,https://doi.org/10.48550/arXiv.2202.07179,arXiv:2202.07179,76,57,141,MLRC,2022,https://openreview.net/forum?id=XxUIomN-ndH,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,0.43,8,Not given,3.5/8,Results — Due to many missing implementation d...
1,RS_002_MLRC_2022_02,[Re] Exact Feature Distribution Matching for A...,"['Mert Erkol', 'Furkan KÄ±nlÄ±', 'BarÄ±ÅŸ Ã–zc...",Replication,"['feature distribution matching', 'style trans...",Reproducibility Summary:\n\nIn this reproducib...,/pdf/7bcb577c2a46db29c48234a5b72368053c7ebed3.pdf,NaN,https://openaccess.thecvf.com/content/CVPR2022...,https://www.doi.org/10.5281/zenodo.8173652,NaN,10.1109/CVPR52688.2022.00787,10.1109/CVPR52688.2022.00787,65,38,160,MLRC,2022,https://openreview.net/forum?id=a5_hbZf0NB&not...,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,5,Not given,Although our experimental results are not\nide...,Results — We have reproduced the experiments d...
2,RS_003_MLRC_2022_03,[Re] End-to-end Algorithm Synthesis with Recur...,"['Sean Michael McLeish', 'Long Tran-Thanh']",Replication,"['rescience c', 'Algorithmic Reasoning', 'Deep...","Scope of Reproducibility:\nIn this report, we ...",/pdf/07d5d68b5873d779bd1fd8c95b9767cb57fe0bc4.pdf,NaN,https://arxiv.org/pdf/2202.05826.pdf,https://www.doi.org/10.5281/zenodo.8173654,NaN,https://doi.org/10.48550/arXiv.2202.05826,arXiv:2202.05826,15,15,34,MLRC,2022,https://openreview.net/forum?id=WaZB4pUVTi,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,3,Not given,apart from perturbation testing for which we p...,Results — We verify the authors’ claims by rep...
3,RS_004_MLRC_2022_04,[Re] Label-Free Explainability for Unsupervise...,"['Eric Langezaal', 'Jesse Belleman', 'Joeri No...",Replication,"['Reproducibility', 'label-free', 'unsupervise...",Scope of Reproducibility â€” This study is an ...,/pdf/7fde4f12c675150699fc376cd097d5b9dad0b4d4.pdf,NaN,https://proceedings.mlr.press/v162/crabbe22a.html,https://www.doi.org/10.5281/zenodo.8173656,NaN,https://doi.org/10.48550/arXiv.2203.01928,arXiv:2203.01928,12,6,16,MLRC,2022,https://openreview.net/forum?id=bBVZ3pY4z8p,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,3,Not given,"Our results differ in some minor aspects, but ...",Results — We find that the three main claims o...
4,RS_005_MLRC_2022_05,[Re] Exploring the Representation of Word Mean...,"['Matteo Brivio', 'Cagri Coltekin']",Replication,"['BERT', 'fastText', 'embeddings', 'language m...",This report summarizes our efforts to reproduc...,/pdf/78430c5af33bc892d852b49a5a6e93abeb314c6f.pdf,NaN,https://aclanthology.org/2021.acl-long.281/,https://www.doi.org/10.5281/zenodo.8173658,NaN,10.18653/v1/2021.acl-long.281,10.18653/v1/2021.acl-long.281,7,6,10,MLRC,2022,https://openreview.net/forum?id=Od5dD58libt,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,0.75,4,Not given,We only manage to partially reproduce the orig...,Results — We only manage to partially reproduc...
5,RS_006_MLRC_2022_06,[Re] Intriguing Properties of Contrastive Losses,"['Luca Marini', 'Mohamad Nabeel', 'Alexandre L...",Replication,"['self-supervised learning', 'contrastive lear...",Reproducibility Summary\n\nScope of Reproducib...,/pdf

In [21]:
for index, row in df_doi.iterrows():
    
    if str(row['doi_org']) == 'nan':
        s2ga_key = "NOT_RELEVANT"
        
        df_doi.at[index, 's2ga_key'] = s2ga_key
        
        print(s2ga_key)
        print('\n\n\n')
        continue
        
    s2ga_key = row['doi_org']
    
    print(row['key_for_all_RS'])
    raw_doi_org = row['doi_org']
    print(raw_doi_org)
    
    if 'doi.org' in raw_doi_org.lower() and 'arxiv' in raw_doi_org.lower():
        new_doi = raw_doi_org.rpartition('/')[-1]
        print(new_doi)
        
        s2ga_key = new_doi.replace('.',':',1).strip()
        print('s2ga_key :',s2ga_key)
        
        df_doi.at[index, 's2ga_key'] = s2ga_key

        print('\n\n\n')
        continue
        
    if 'doi.org' in raw_doi_org.lower():
        new_doi = raw_doi_org.partition('doi.org/')[-1]
        print(new_doi)
        
        s2ga_key = 'DOI:'+new_doi
        
        df_doi.at[index, 's2ga_key'] = s2ga_key
        
        print('s2ga_key :',s2ga_key)
        print('\n\n\n')
        
        continue

    if 'arxiv' in raw_doi_org.lower():
        new_doi = raw_doi_org.partition('/')[-1]
        print(new_doi)
        
        s2ga_key = new_doi.replace('.',':',1).strip()
        
        
        df_doi.at[index, 's2ga_key'] = s2ga_key
        
        print('s2ga_key :',s2ga_key)
        print('\n\n\n')
        
        continue
        
    if "corpus id" in raw_doi_org.lower():
        new_doi = raw_doi_org.lower().replace('corpus id:','').strip()
        print(new_doi)

        s2ga_key = 'CorpusId:'+new_doi
        
        df_doi.at[index, 's2ga_key'] = s2ga_key
        
        print('s2ga_key :',s2ga_key)
        
        print('\n\n\n')
        continue
    
    df_doi.at[index, 's2ga_key'] = s2ga_key
    
    print('s2ga_key :',s2ga_key)
    print('\n\n\n')

    
df_doi.to_excel(r"../data/RS_ALL_IN_ONE_metadata_with_s2ga_key_9_21.xlsx",index=False)

RS_001_MLRC_2022_01
https://doi.org/10.48550/arXiv.2202.07179
arXiv.2202.07179
s2ga_key : arXiv:2202.07179




RS_002_MLRC_2022_02
10.1109/CVPR52688.2022.00787
s2ga_key : 10.1109/CVPR52688.2022.00787




RS_003_MLRC_2022_03
https://doi.org/10.48550/arXiv.2202.05826
arXiv.2202.05826
s2ga_key : arXiv:2202.05826




RS_004_MLRC_2022_04
https://doi.org/10.48550/arXiv.2203.01928
arXiv.2203.01928
s2ga_key : arXiv:2203.01928




RS_005_MLRC_2022_05
10.18653/v1/2021.acl-long.281
s2ga_key : 10.18653/v1/2021.acl-long.281




RS_006_MLRC_2022_06
https://doi.org/10.48550/arXiv.2011.02803
arXiv.2011.02803
s2ga_key : arXiv:2011.02803




RS_007_MLRC_2022_07
https://doi.org/10.48550/arXiv.2206.02092
arXiv.2206.02092
s2ga_key : arXiv:2206.02092




RS_008_MLRC_2022_08
10.1109/CVPR52688.2022.00031
s2ga_key : 10.1109/CVPR52688.2022.00031




RS_009_MLRC_2022_09
https://doi.org/10.48550/arXiv.2205.10279
arXiv.2205.10279
s2ga_key : arXiv:2205.10279




RS_010_MLRC_2022_10
10.1007/978-3-031-19775-8_26
s2ga

In [45]:
len(df_doi)

149

In [46]:
df_rel = df_doi[df_doi['article_type']!='Editorial']
len(df_rel)

145

In [47]:
for col in df_rel.columns:
    print(col)

key_for_all_RS
title_rs
authors_rs
article_type
keywords
abstract
pdf_url_rs
code_url_rs
org_paper_url
doi_rs
code_doi_rs
doi_org
s2ga_key
venue_name_rs
challenge_year_rs
review_url_rs
journal_rs
volume_journal_rs
issue_journal_rs
published_year_journal_rs
domain
entry_type
rs_score
rs_score_comment
rs_comment


## ``processed doi for S2GA``

In [23]:
df_doi = pd.read_excel(r"../data/RS_ALL_IN_ONE_metadata.xlsx",sheet_name="Sheet1")

df_doi.head()

,key_for_all_RS,title_rs,authors_rs,article_type,keywords,abstract,pdf_url_rs,code_url_rs,org_paper_url,doi_rs,code_doi_rs,doi_org,s2ga_key,citation_count_org,citing_paper_count,total_context_count_for_labelling,venue_name_rs,challenge_year_rs,review_url_rs,journal_rs,volume_journal_rs,issue_journal_rs,published_year_journal_rs,domain,entry_type,rs_score,total_claims_or_attempts,results_threshold,rs_score_comment,rs_comment
0,RS_001_MLRC_2022_01,[Re] $\mathcal{G}$-Mixup: Graph Data Augmentat...,"['Ermin Omeragic', 'Vuk ÄuranoviÄ‡']",Replication,"['rescience c', 'machine learning', 'data augm...",Scope of Reproducibility\nThis paper presents ...,/pdf/8c87cb1e84e1482826c40a3b0c43928eaef747f3.pdf,NaN,https://proceedings.mlr.press/v162/han22c.html,https://www.doi.org/10.5281/zenodo.8173650,NaN,https://doi.org/10.48550/arXiv.2202.07179,arXiv:2202.07179,76,57,141,MLRC,2022,https://openreview.net/forum?id=XxUIomN-ndH,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,0.43,8,Not given,3.5/8,Results — Due to many missing implementation d...
1,RS_002_MLRC_2022_02,[Re] Exact Feature Distribution Matching for A...,"['Mert Erkol', 'Furkan KÄ±nlÄ±', 'BarÄ±ÅŸ Ã–zc...",Replication,"['feature distribution matching', 'style trans...",Reproducibility Summary:\n\nIn this reproducib...,/pdf/7bcb577c2a46db29c48234a5b72368053c7ebed3.pdf,NaN,https://openaccess.thecvf.com/content/CVPR2022...,https://www.doi.org/10.5281/zenodo.8173652,NaN,10.1109/CVPR52688.2022.00787,10.1109/CVPR52688.2022.00787,65,38,160,MLRC,2022,https://openreview.net/forum?id=a5_hbZf0NB&not...,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,5,Not given,Although our experimental results are not\nide...,Results — We have reproduced the experiments d...
2,RS_003_MLRC_2022_03,[Re] End-to-end Algorithm Synthesis with Recur...,"['Sean Michael McLeish', 'Long Tran-Thanh']",Replication,"['rescience c', 'Algorithmic Reasoning', 'Deep...","Scope of Reproducibility:\nIn this report, we ...",/pdf/07d5d68b5873d779bd1fd8c95b9767cb57fe0bc4.pdf,NaN,https://arxiv.org/pdf/2202.05826.pdf,https://www.doi.org/10.5281/zenodo.8173654,NaN,https://doi.org/10.48550/arXiv.2202.05826,arXiv:2202.05826,15,15,34,MLRC,2022,https://openreview.net/forum?id=WaZB4pUVTi,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,3,Not given,apart from perturbation testing for which we p...,Results — We verify the authors’ claims by rep...
3,RS_004_MLRC_2022_04,[Re] Label-Free Explainability for Unsupervise...,"['Eric Langezaal', 'Jesse Belleman', 'Joeri No...",Replication,"['Reproducibility', 'label-free', 'unsupervise...",Scope of Reproducibility â€” This study is an ...,/pdf/7fde4f12c675150699fc376cd097d5b9dad0b4d4.pdf,NaN,https://proceedings.mlr.press/v162/crabbe22a.html,https://www.doi.org/10.5281/zenodo.8173656,NaN,https://doi.org/10.48550/arXiv.2203.01928,arXiv:2203.01928,12,6,16,MLRC,2022,https://openreview.net/forum?id=bBVZ3pY4z8p,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,1,3,Not given,"Our results differ in some minor aspects, but ...",Results — We find that the three main claims o...
4,RS_005_MLRC_2022_05,[Re] Exploring the Representation of Word Mean...,"['Matteo Brivio', 'Cagri Coltekin']",Replication,"['BERT', 'fastText', 'embeddings', 'language m...",This report summarizes our efforts to reproduc...,/pdf/78430c5af33bc892d852b49a5a6e93abeb314c6f.pdf,NaN,https://aclanthology.org/2021.acl-long.281/,https://www.doi.org/10.5281/zenodo.8173658,NaN,10.18653/v1/2021.acl-long.281,10.18653/v1/2021.acl-long.281,7,6,10,MLRC,2022,https://openreview.net/forum?id=Od5dD58libt,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,0.75,4,Not given,We only manage to partially reproduce the orig...,Results — We only manage to partially reproduc...


In [29]:
df_doi = pd.read_excel(r"../data/RS_ALL_IN_ONE_metadata.xlsx",sheet_name="Sheet1")

FOLDER_NAME = r'../data/Citing_Paper_details/'

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)


for index, row in df_doi.iterrows():
    
    if 'NOT_RE' in row['s2ga_key']:
        continue
    
    url = "https://api.semanticscholar.org/graph/v1/paper/{s2ga_key}?fields=title,authors,citationCount,citations.title,citations.authors,citations.year,citations.publicationVenue,citations.corpusId,citations.url,citations.externalIds".format(s2ga_key=row['s2ga_key'])
    
    print(row['key_for_all_RS'])
    print(url)
    
    page = requests.get(url)
    
    json_object = json.dumps(page.json(), indent=4)
    
    print(page.status_code)
    
    file_name = row['key_for_all_RS']+'.json'
    out_file_name = os.path.join(FOLDER_NAME, file_name)
    
    
    with open(out_file_name, "w") as outfile:
        outfile.write(json_object)
    
    time.sleep(10)
    
    print('\n\n\n')
    
    if index == 5:
        break


RS_001_MLRC_2022_01
https://api.semanticscholar.org/graph/v1/paper/arXiv:2202.07179?fields=title,authors,citationCount,citations.title,citations.authors,citations.year,citations.publicationVenue,citations.corpusId,citations.url,citations.externalIds
200




RS_002_MLRC_2022_02
https://api.semanticscholar.org/graph/v1/paper/10.1109/CVPR52688.2022.00787?fields=title,authors,citationCount,citations.title,citations.authors,citations.year,citations.publicationVenue,citations.corpusId,citations.url,citations.externalIds
200




RS_003_MLRC_2022_03
https://api.semanticscholar.org/graph/v1/paper/arXiv:2202.05826?fields=title,authors,citationCount,citations.title,citations.authors,citations.year,citations.publicationVenue,citations.corpusId,citations.url,citations.externalIds
200




RS_004_MLRC_2022_04
https://api.semanticscholar.org/graph/v1/paper/arXiv:2203.01928?fields=title,authors,citationCount,citations.title,citations.authors,citations.year,citations.publicationVenue,citations.corpusId,

In [31]:
#RS_SCORE distribution
df_excel = pd.read_excel(r"../data/RS_ALL_IN_ONE_metadata.xlsx",sheet_name="Sheet1")
print(len(df_excel['rs_score']))
len(df_excel[df_excel['rs_score']!='NOT_RELEVANT']['rs_score'].astype(float))

149


145

## ``Collecting Citations Contexts from "S2GA"``

In [35]:
df_doi = pd.read_excel(r"../data/RS_ALL_IN_ONE_metadata.xlsx",sheet_name="Sheet1")

FOLDER_NAME = r"../data/Citing_Paper_contexts/"

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)

for index, row in df_doi.iterrows():
    
    if 'NOT_RE' in row['s2ga_key']:
        continue
    
    url = "https://api.semanticscholar.org/graph/v1/paper/{s2ga_key}/citations?fields=contexts,intents,isInfluential,abstract,title,year,publicationVenue,corpusId,externalIds,url,authors&offset=0&limit=1000".format(s2ga_key=row['s2ga_key'])
    
    page = requests.get(url)
    all_data = page.json()

    limit = 1000

    # handling 1000 retrieval limitation per request
    if row['citation_count_org']>1000:
        for index in range(0,int(row['citation_count_org']/1000)):
            time.sleep(3)
            step = index+1
            print(index,'--', step)
            # 1001 - 2000
            print('offset',(step*limit+1),'until:',(step+1)*1000)

            offset_ = step*limit # step*limit+1
            # limit_ = (step+1)*1000

            url = "https://api.semanticscholar.org/graph/v1/paper/10.1109/CVPR52688.2022.01553/citations?fields=contexts,intents,isInfluential,abstract,title,year,publicationVenue,corpusId,externalIds,url,authors&offset={offset}&limit={limit}".format(offset=offset_,limit=limit)
            print(url)

            page = requests.get(url)
            new_page_ = page.json()
            print(new_page_)

            all_data['data'].extend(new_page_['data']) 
            print("len(all_data['data']):",len(all_data['data']))

    json_object = json.dumps(all_data, indent=4)

    print(row['key_for_all_RS'])
    print("citation_count_org :",row['citation_count_org'])
    if page.status_code == 200:
        print("len(all_data['data']) :",len(all_data['data']))
    print(url)
    print('page.status_code: ',page.status_code)
    
    file_name = row['key_for_all_RS']+'.json'
    out_file_name = os.path.join(FOLDER_NAME, file_name)
    
    if index == 3:
        break
    
    
    with open(out_file_name, "w") as outfile:
        outfile.write(json_object)
    
    time.sleep(10)
    
    print('\n\n\n')

RS_001_MLRC_2022_01
citation_count_org : 76
https://api.semanticscholar.org/graph/v1/paper/arXiv:2202.07179/citations?fields=contexts,intents,isInfluential,abstract,title,year,publicationVenue,corpusId,externalIds,url,authors&offset=0&limit=1000
page.status_code:  429




RS_002_MLRC_2022_02
citation_count_org : 65
len(all_data['data']) : 94
https://api.semanticscholar.org/graph/v1/paper/10.1109/CVPR52688.2022.00787/citations?fields=contexts,intents,isInfluential,abstract,title,year,publicationVenue,corpusId,externalIds,url,authors&offset=0&limit=1000
page.status_code:  200




RS_003_MLRC_2022_03
citation_count_org : 15
len(all_data['data']) : 17
https://api.semanticscholar.org/graph/v1/paper/arXiv:2202.05826/citations?fields=contexts,intents,isInfluential,abstract,title,year,publicationVenue,corpusId,externalIds,url,authors&offset=0&limit=1000
page.status_code:  200




RS_004_MLRC_2022_04
citation_count_org : 12
len(all_data['data']) : 15
https://api.semanticscholar.org/graph/v1/pap

In [37]:
df_excel_ = pd.read_excel(r"../data/RS_ALL_IN_ONE_metadata.xlsx",sheet_name="Sheet1")
df_excel_

df_excel_["citing_paper_count"]

0      57
1      38
2      15
3       6
4       6
       ..
144    28
145     0
146    59
147     2
148     2
Name: citing_paper_count, Length: 149, dtype: int64